In [1]:
const int L = 5;            // average parameter
const int interval = 100;   // for standard waveform length and pileup
const int rangeuseleft = -15;
const int rangeuseright = 75;
int rangeleft = 0;
int rangeright = 2*interval;

In [2]:
%jsroot on
TFile *ipf1 = new TFile("../../data/standardwave_0314_W200.root");
TGraph *gstd = (TGraph*)ipf1->Get("waveLaBr3_0");
TCanvas *c1 = new TCanvas;
TRandom3 *gr = new TRandom3(0);

Warning in <TClass::Init>: no dictionary for class dets is available


In [3]:
double ffit(double *val, double *par)
{
    double x = val[0];
    int npeaks = par[0];
    
    double wave = 0;
    for (int ipeak = 0; ipeak < npeaks; ipeak++){
        double A = par[3*ipeak+1];
        double pos = par[3*ipeak+2];
        //int type = par[3*ipeak+3];
        
        if ( x - pos < rangeuseleft || x - pos > rangeuseright )
            wave += 0;
        else
            wave += A * gstd->Eval(x - pos + interval);
    }
    return wave;
}

In [4]:
const int npeaks0 = 4;
const double noise = 6;

In [5]:
TF1 *f0 = new TF1("f0", ffit, 0, 2*interval, 3*npeaks0+1);
double par0[] = {npeaks0, gr->Uniform(500, 5000), gr->Uniform(50, 100), 0, 
                          gr->Uniform(500, 5000), gr->Uniform(50, 100), 0, 
                          gr->Uniform(500, 5000), gr->Uniform(50, 100), 0, 
                          gr->Uniform(500, 5000), gr->Uniform(50, 100), 0};
f0->SetParameters(par0);

In [6]:
TGraph *gspe = new TGraph;
for (int ipnt = rangeleft; ipnt < rangeright; ipnt++){
    double Noise;
    do {
        Noise = gr->Gaus(0, noise);
    } while (abs(Noise) > 20);
    gspe->SetPoint( ipnt, ipnt, f0->Eval(ipnt) + Noise );
}

In [7]:
TH1D *hspe = new TH1D("hspe", "hspe", rangeright-rangeleft, rangeleft, rangeright);
for (int ipnt = rangeleft; ipnt < rangeright; ipnt++)
    hspe->SetBinContent(ipnt+1, gspe->GetPointY(ipnt));
TSpectrum *s = new TSpectrum(500);
int nfound = s->Search(hspe, 2, "", 0.05);
Double_t *xpeaks = s->GetPositionX();
vector<int> vmaxpnt;
for (int ipeak = 0; ipeak < nfound; ipeak++)
    vmaxpnt.push_back(xpeaks[ipeak]);

In [8]:
TF1 *f;
double maxdiff = 0, mindiff = 0;
int maxpnt = -1, minpnt = -1;
double chi2ndf = -1;
bool firstfit = true;

In [9]:
do {
    int npeaks = vmaxpnt.size();
    
    f = new TF1("ffit", ffit, rangeleft, rangeright, 3*npeaks+1);
    f->SetNpx(rangeright-rangeleft);

    f->FixParameter(0, npeaks);
    for (int ipeak = 0; ipeak < npeaks; ipeak++){
        f->SetParameter(3*ipeak+1, gspe->GetPointY(vmaxpnt[ipeak]));
        f->SetParLimits(3*ipeak+1, 0, 2*gspe->GetPointY(vmaxpnt[ipeak]));
        f->SetParameter(3*ipeak+2, vmaxpnt[ipeak]);
        f->SetParLimits(3*ipeak+2, 0, 2*interval);
        f->FixParameter(3*ipeak+3, 0);
    }

    TFitResultPtr fr = gspe->Fit(f, "SR", "", rangeleft, rangeright);
    chi2ndf = fr->Chi2() / fr->Ndf();

    maxdiff = 0;
    maxpnt = -1;
    mindiff = 0;
    minpnt = -1;
    for (int ipnt = rangeleft; ipnt < rangeright; ipnt++){
        double diff = ( gspe->GetPointY(ipnt) - f->Eval(ipnt) );
        if (diff > maxdiff){
            maxpnt = ipnt;
            maxdiff = diff;
        }
        if (diff < mindiff){
            minpnt = ipnt;
            mindiff = diff;
        }
    }
                    
    bool repeat = false;
    for (int imaxpnt : vmaxpnt)
        if ( abs(maxpnt-imaxpnt) <= 2 ){
            repeat = true;
            break;
        }
                    
    if (!repeat)
        vmaxpnt.push_back(maxpnt);
    else
        vmaxpnt.push_back(minpnt);
} while (chi2ndf > 50 && int(vmaxpnt.size()) < nfound + 5);


****************************************
         Invalid FitResult  (status = 4 )
****************************************
Minimizer is Minuit / Migrad
Chi2                      =  4.54837e+06
NDf                       =          196
Edm                       =       159325
NCalls                    =          165
p0                        =            2                      	 (fixed)
p1                        =      5578.83   +/-   41.1446      	 (limited)
p2                        =       58.373   +/-   0.0734895    	 (limited)
p3                        =            0                      	 (fixed)
p4                        =      2516.18   +/-   41.1236      	 (limited)
p5                        =           98   +/-   0.000273009  	 (limited)
p6                        =            0                      	 (fixed)

****************************************
Minimizer is Minuit / Migrad
Chi2                      =      7800.63
NDf                       =          194
Edm              

Warning in <Fit>: Abnormal termination of minimization.


In [10]:
gspe->Draw();
c1->Draw();